<a href="https://colab.research.google.com/github/percoguru/FakeNewstfidf/blob/master/FakeNewsClassifiertfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
cd My Drive/Dataset

/content/drive/My Drive/Dataset


In [0]:
from fastai import *
from fastai.text import *
from pathlib import Path
import pandas as pd
import numpy as np
import re
import os

In [0]:
fake_news = pd.read_csv("fake.csv")
real_news = pd.read_csv("real_news.csv")

In [14]:
#Here are the size of our datasets:
print(fake_news.shape)
print(real_news.shape)

(12999, 20)
(15712, 11)


In [15]:
# Let's see what columns we have
print(list(fake_news.columns))
print(list(real_news.columns))

['uuid', 'ord_in_thread', 'author', 'published', 'title', 'text', 'language', 'crawled', 'site_url', 'country', 'domain_rank', 'thread_title', 'spam_score', 'main_img_url', 'replies_count', 'participants_count', 'likes', 'comments', 'shares', 'type']
['Unnamed: 0', 'Unnamed: 0.1', 'id', 'title', 'publication', 'author', 'date', 'year', 'month', 'url', 'content']


In [27]:
# now let's obtain similar features for both datasets before we combine them
# Let's add our label to the dataset "real" for real news and "fake" for fake news

real_news2 = real_news[['title', 'author', 'content']]
real_news2['label'] = 'real'
real_news2.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,title,author,content,label
0,House Republicans Fret About Winning Their Hea...,Carl Hulse,WASHINGTON — Congressional Republicans have...,real
1,"First, a Mixtape. Then a Romance. - The New Yo...",Katherine Rosman,"Just how is Hillary Kerr, the founder of ...",real
2,Calling on Angels While Enduring the Trials of...,Andy Newman,Angels are everywhere in the Muñiz family’s ap...,real
3,U.S. Plans to Step Up Military Campaign Agains...,Michael S. Schmidt and Eric Schmitt,"ABU DHABI, United Arab Emirates — The Obama...",real
4,272 Slaves Were Sold to Save Georgetown. What ...,Rachel L. Swarns,WASHINGTON — The human cargo was loaded on ...,real
5,"Among Travelers and Commuters, the Homeless St...",Corey Kilgannon,Wilson Silva said he knew the homeless situati...,real
6,Bus Bombing in Jerusalem Wounds 21 - The New Y...,Isabel Kershner,JERUSALEM — A bomb exploded on a bus in Jer...,real
7,Syria Cease-Fire Crumbles as Bombings Kill Doz...,Anne Barnard,"BEIRUT, Lebanon — For 38 straight days, the...",real
8,Iceland’s Water Cure - The New York Times,Dan Kois,"On a frigid February day in Reykjavik, I stood...",real
9,"Get to Know the Historical Figures on the $5, ...",Julie Hirschfeld Davis and Jada F. Smith,WASHINGTON — The Treasury Department announ...,real


In [28]:
fake_news2 = fake_news[['title', 'author', 'text']]
fake_news2['label'] = 'fake'
fake_news2.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,title,author,text,label
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Barracuda Brigade,Print They should pay all the back all the mon...,fake
1,Re: Why Did Attorney General Loretta Lynch Ple...,reasoning with facts,Why Did Attorney General Loretta Lynch Plead T...,fake
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Barracuda Brigade,Red State : \nFox News Sunday reported this mo...,fake
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Fed Up,Email Kayla Mueller was a prisoner and torture...,fake
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Fed Up,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,fake
5,Hillary Goes Absolutely Berserk On Protester A...,Barracuda Brigade,Print Hillary goes absolutely berserk! She exp...,fake
6,BREAKING! NYPD Ready To Make Arrests In Weiner...,Fed Up,BREAKING! NYPD Ready To Make Arrests In Weiner...,fake
7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,Fed Up,BREAKING! NYPD Ready To Make Arrests In Weiner...,fake
8,BREAKING: CLINTON CLEARED...Was This A Coordin...,Fed Up,\nLimbaugh said that the revelations in the Wi...,fake
9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",Fed Up,Email \nThese people are sick and evil. They w...,fake


In [29]:
# let's rename the features in our datasets to be the same
newlabels = ['title', 'author', 'content', 'label']
real_news2.columns = newlabels
fake_news2.columns = newlabels

# let's concatenate the dataframes
frames = [fake_news2, real_news2]
news_dataset = pd.concat(frames)
news_dataset

,title,author,content,label
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Barracuda Brigade,Print They should pay all the back all the mon...,fake
1,Re: Why Did Attorney General Loretta Lynch Ple...,reasoning with facts,Why Did Attorney General Loretta Lynch Plead T...,fake
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Barracuda Brigade,Red State : \nFox News Sunday reported this mo...,fake
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Fed Up,Email Kayla Mueller was a prisoner and torture...,fake
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Fed Up,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,fake
5,Hillary Goes Absolutely Berserk On Protester A...,Barracuda Brigade,Print Hillary goes absolutely berserk! She exp...,fake
6,BREAKING! NYPD Ready To Make Arrests In Weiner...,Fed Up,BREAKING! NYPD Ready To Make Arrests In Weiner...,fake
7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,Fed Up,BREAKING! NYPD Ready To Make Arrests In Weiner...,fake
8,BREAKING: CLINTON CLEARED...Was This A Coordin...,Fed Up,\nLimbaugh said that the revelations in the Wi...,fake
9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",Fed Up,Email \nThese people are sick and evil. They w...,fake


In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
tf.test.is_gpu_available()

True

In [30]:
news_dataset = news_dataset.sample(frac=1).reset_index(drop=True)
news_dataset.head()

,title,author,content,label
0,Social Media Is Censoring FBI’s Re-Opened Clin...,WashingtonsBlog,"Posted on October 30, 2016 by WashingtonsBlog ...",fake
1,Study Finds Racial Discrimination by Uber and ...,Henry Wolff,Study Finds Racial Discrimination by Uber and ...,fake
2,Why the Orlando Shooting Is Unlikely to Lead t...,Jennifer Steinhauer,WASHINGTON — It didn’t happen after a congr...,real
3,"After an uproar, Trump’s team has disavowed it...",Brad Plumer,The nation’s federal employees appear to have...,real
4,Riots in Portland on Third Night of Protests ...,Ed Krayewski,Riots in Portland on Third Night of Protests ...,fake


In [0]:
#split in samples
from sklearn.model_selection import train_test_split
train, test= train_test_split(news_dataset, random_state=0)


In [35]:
test.info()
test['label']='t'
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7178 entries, 19464 to 13067
Data columns (total 4 columns):
title      6996 non-null object
author     6131 non-null object
content    7166 non-null object
label      7178 non-null object
dtypes: object(4)
memory usage: 280.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21533 entries, 9408 to 2732
Data columns (total 4 columns):
title      21035 non-null object
author     18411 non-null object
content    21499 non-null object
label      21533 non-null object
dtypes: object(4)
memory usage: 841.1+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
train = Train
test = Test

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

#data prep
test=test.fillna(' ')
train=train.fillna(' ')
test['total']=test['title']+' '+test['author']+test['content']
train['total']=train['title']+' '+train['author']+train['content']

#tfidf
transformer = TfidfTransformer(smooth_idf=False)
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
counts = count_vectorizer.fit_transform(train['total'].values)
tfidf = transformer.fit_transform(counts)


targets = train['label'].values
test_counts = count_vectorizer.transform(test['total'].values)
test_tfidf = transformer.fit_transform(test_counts)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1278: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [0]:
#split in samples
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf, targets, random_state=0)

In [45]:

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(X_train, y_train)
print('Accuracy of Lasso classifier on training set: {:.4f}'
     .format(logreg.score(X_train, y_train)))
print('Accuracy of Lasso classifier on test set: {:.4f}'
     .format(logreg.score(X_test, y_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of Lasso classifier on training set: 1.0000
Accuracy of Lasso classifier on test set: 0.9528
